In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
dir_path = '/content/drive/Othercomputers/macbook-air/TPS-Feb2022/notebooks'
os.chdir(dir_path)

In [3]:
! pip install japanize-matplotlib
! pip install shap

     |████████████████████████████████| 4.1 MB 5.0 MB/s 
  Created wheel for japanize-matplotlib: filename=japanize_matplotlib-1.1.3-py3-none-any.whl size=4120275 sha256=b18ce3c546a0cd24b14d936ce326dc8687388630b8a67205fe8795f02e9c422c
  Stored in directory: /root/.cache/pip/wheels/83/97/6b/e9e0cde099cc40f972b8dd23367308f7705ae06cd6d4714658
Successfully built japanize-matplotlib
     |████████████████████████████████| 564 kB 5.1 MB/s 


In [4]:
import glob
import sys,os
import json
import pprint
import time
import re
import datetime
import pickle
import string
import gc
import warnings
import yaml
import os
warnings.filterwarnings("ignore")
sys.path.append(os.pardir)
sys.path.append('../..')
sys.path.append('../../..')

import numpy as np
import pandas as pd
import pandas_profiling as pdp
import matplotlib.pyplot as plt
import japanize_matplotlib # 日本語対応
import seaborn as sns
# pandasのオプション
pd.set_option('display.max_columns', 300)
pd.set_option('display.max_rows', 300)
pd.set_option('display.max_colwidth', 5000)
pd.options.display.float_format = '{:.3f}'.format
%matplotlib inline
# sns.set_style('whitegrid')
plt.style.use('fivethirtyeight')

from joblib import Parallel, delayed # よりお手軽にサクっと並列処理を実行出来るモジュール
from tqdm import tqdm, tqdm_notebook # プログレスバーを表示できる
from PIL import Image
tqdm.pandas()

# 外部モジュールを自動的にリロードする
%load_ext autoreload
%autoreload 2

## datasets確認

In [5]:
CONFIG_FILE = '../configs/config.yaml'

with open(CONFIG_FILE) as file:
    yml = yaml.load(file)
MODEL_DIR_NAME = yml['SETTING']['MODEL_DIR_NAME']
FEATURE_DIR_NAME = yml['SETTING']['FEATURE_DIR_NAME']

In [6]:
def load_datasets_train(feats):
    dfs = [pd.read_pickle(FEATURE_DIR_NAME + f'{f}_train.pkl') for f in feats]
    X_train = pd.concat(dfs, axis=1)
    return X_train

def load_datasets_both(feats):
    dfs = [pd.read_pickle(FEATURE_DIR_NAME + f'{f}_train.pkl') for f in feats]
    X_train = pd.concat(dfs, axis=1)
    dfs = [pd.read_pickle(FEATURE_DIR_NAME + f'{f}_test.pkl') for f in feats]
    X_test = pd.concat(dfs, axis=1)
    return X_train, X_test

# 欠損値の確認
def missing_values_table(data):
    total = data.isnull().sum()
    percent = (data.isnull().sum()/data.isnull().count()*100)
    tt = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    types = []
    for col in data.columns:
        dtype = str(data[col].dtype)
        types.append(dtype)
    tt['Types'] = types
    return(np.transpose(tt))

In [7]:
features = [
'pca_output'
]

## モデルデータ確認

In [10]:
pd.read_pickle(MODEL_DIR_NAME + 'lgb_0222_0016/lgb_0222_0016-proba.pkl')

,0,1,2,3,4,5,6,7,8,9
0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000
1,0.000,0.999,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000
2,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
3,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
4,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000
...,...,...,...,...,...,...,...,...,...,...
99995,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000
99996,0.000,0.002,0.001,0.001,0.000,0.004,0.000,0.003,0.990,0.000
99997,0.044,0.003,0.000,0.023,0.000,0.053,0.000,0.012,0.865,0.000
99998,0.000,0.004,0.003,0.000,0.000,0.018,0.003,0.000,0.971,0.000


In [ ]:
display(train.shape, test.shape)

In [ ]:
plt.bar([n for n in range(1, len(pca.explained_variance_ratio_)+1)], pca.explained_variance_ratio_)

In [ ]:
# display(train.head(), train.tail(), train.shape)
# len(train[train['pca4'] > 0.01])
train.max()

pca0     0.085
pca1     0.092
pca2     0.069
pca3     0.054
pca4     0.057
pca5     0.043
pca6     0.050
pca7     0.057
pca8     0.035
pca9     0.057
pca10    0.050
pca11    0.052
pca12    0.061
pca13    0.047
pca14    0.053
pca15    0.050
pca16    0.054
pca17    0.048
pca18    0.045
pca19    0.053
pca20    0.047
pca21    0.054
pca22    0.038
pca23    0.039
pca24    0.039
pca25    0.043
pca26    0.041
pca27    0.042
pca28    0.038
pca29    0.039
pca30    0.040
pca31    0.038
pca32    0.032
pca33    0.037
pca34    0.034
pca35    0.038
pca36    0.039
pca37    0.036
pca38    0.037
pca39    0.038
pca40    0.040
pca41    0.035
pca42    0.034
pca43    0.035
pca44    0.038
pca45    0.036
pca46    0.034
pca47    0.032
pca48    0.033
pca49    0.030
pca50    0.028
pca51    0.032
pca52    0.032
pca53    0.040
pca54    0.031
pca55    0.028
pca56    0.031
pca57    0.039
pca58    0.032
pca59    0.029
pca60    0.028
pca61    0.023
pca62    0.031
pca63    0.033
pca64    0.027
pca65    0.026
pca66    0

In [ ]:
display(train.describe(), test.describe())

,Age,Age_mis_val_median,Family_Size,Fare,Fare_mis_val_median,Parch,PassengerId,Pclass,sex_label_encoder,SibSp,Survived
count,714.000,891.000,891.000,891.000,891.000,891.000,891.000,891.000,891.000,891.000,891.000
mean,29.699,29.362,0.905,32.204,32.204,0.382,446.000,2.309,0.648,0.523,0.384
std,14.526,13.020,1.613,49.693,49.693,0.806,257.354,0.836,0.478,1.103,0.487
min,0.420,0.420,0.000,0.000,0.000,0.000,1.000,1.000,0.000,0.000,0.000
25%,20.125,22.000,0.000,7.910,7.910,0.000,223.500,2.000,0.000,0.000,0.000
50%,28.000,28.000,0.000,14.454,14.454,0.000,446.000,3.000,1.000,0.000,0.000
75%,38.000,35.000,1.000,31.000,31.000,0.000,668.500,3.000,1.000,1.000,1.000
max,80.000,80.000,10.000,512.329,512.329,6.000,891.000,3.000,1.000,8.000,1.000


,Age,Age_mis_val_median,Family_Size,Fare,Fare_mis_val_median,Parch,PassengerId,Pclass,sex_label_encoder,SibSp
count,332.000,418.000,418.000,417.000,418.000,418.000,418.000,418.000,418.000,418.000
mean,30.273,29.599,0.840,35.627,35.577,0.392,1100.500,2.266,0.636,0.447
std,14.181,12.704,1.519,55.908,55.850,0.981,120.810,0.842,0.482,0.897
min,0.170,0.170,0.000,0.000,0.000,0.000,892.000,1.000,0.000,0.000
25%,21.000,23.000,0.000,7.896,7.896,0.000,996.250,1.000,0.000,0.000
50%,27.000,27.000,0.000,14.454,14.454,0.000,1100.500,3.000,1.000,0.000
75%,39.000,35.750,1.000,31.500,31.472,0.000,1204.750,3.000,1.000,1.000
max,76.000,76.000,10.000,512.329,512.329,9.000,1309.000,3.000,1.000,8.000


In [ ]:
# 各データの欠損値を確認
display(
    missing_values_table(train),
    missing_values_table(test)
)

,Age,Age_mis_val_median,Cabin,Embarked,Family_Size,Fare,Fare_mis_val_median,Name,Parch,PassengerId,Pclass,Sex,sex_label_encoder,SibSp,Survived,Ticket
Total,177,0,687,2,0,0,0,0,0,0,0,0,0,0,0,0
Percent,19.865,0.000,77.104,0.224,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Types,float64,float64,object,object,int64,float64,float64,object,int64,int64,int64,object,int64,int64,int64,object


,Age,Age_mis_val_median,Cabin,Embarked,Family_Size,Fare,Fare_mis_val_median,Name,Parch,PassengerId,Pclass,Sex,sex_label_encoder,SibSp,Ticket
Total,86,0,327,0,0,1,0,0,0,0,0,0,0,0,0
Percent,20.574,0.000,78.230,0.000,0.000,0.239,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Types,float64,float64,object,object,int64,float64,float64,object,int64,int64,int64,object,int64,int64,object


## create submission

In [ ]:
warnings.filterwarnings("ignore")

In [ ]:
! python ../code/20_run.py

特徴量ディレクトリ:../data/features/ で実行しますか？[Y/n]
>> Y
[2022-02-06 03:15:34] - lgb_0206_0314 - train_x shape: (891, 7)
[2022-02-06 03:15:34] - lgb_0206_0314 - train_y shape: (891,)
[2022-02-06 03:15:34] - lgb_0206_0314 - start training cv
[2022-02-06 03:15:34] - lgb_0206_0314 - cv method: KFold
[2022-02-06 03:15:34] - lgb_0206_0314 fold 0 - start training
[LightGBM] [Warning] Starting from the 2.1.2 version, default value for the "boost_from_average" parameter in "binary" objective is true.
This may cause significantly different results comparing to the previous versions of LightGBM.
Try to set boost_from_average=false, if your old models produce bad results
[LightGBM] [Info] Number of positive: 222, number of negative: 372
[LightGBM] [Info] Total Bins 188
[LightGBM] [Info] Number of data: 594, number of used features: 7
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.373737 -> initscore=-0.516216
[LightGBM] [Info] Start training from score -0.516216
[LightGBM] [Warning] No further splits wi